In [ ]:
import pandas as pd
import numpy as np
import holoviews as hv
from holoviews import opts, dim
import panel as pn
import panel.widgets as pnw
pn.extension()
hv.extension('bokeh')

In [ ]:
#Loading standard results
import numpy as np
import pandas as pd
df0 = pd.read_csv('regions_fadn.csv')

In [ ]:
choose = ['YEAR','REGION','TF14']
cols_bal = ['SE446','SE450','SE455','SE460','SE470','SE475','SE480']
cols_inc = ['SE135', 'SE206', 'SE256', 'SE281', 'SE336', 'SE600','SE360', 'SE370', 'SE375', 'SE380', 'SE420']
cols_com = ['SE011','SE025','SE080','SE131','SE132','SE135','SE136','SE206','SE207','SE256',
            'SE270','SE360','SE441','SE455','SE420','SE605']
oles = list(set(choose + cols_bal + cols_inc + cols_com + ['FADN_2012_']))
negs = ['SE281', 'SE336', 'SE275', 'SE370', 'SE360', 'SE375', 'SE380', 'SE365']
names = {'SE011':'Total Hours', 'SE025': 'Total Area', 'SE080': 'Livestock Units', 'SE131': 'Total Output',
         'SE132': 'T.Output/T.Costs', 'SE135': 'Crop Output', 'SE136': 'Cr.Out/Ha', 'SE206': 'Livestock Output',
         'SE207':'Lv.Out/L.Uni', 'SE256': 'Other Output', 'SE270': 'Total Costs', 'SE281': 'Tot.Spec.Costs',
         'SE336': 'Tot.Farm.Overheads', 'SE360': 'Depreciation', 'SE370': 'Wages', 'SE375': 'Rent', 'SE380': 'Interest',
         'SE420': 'Family Income', 'SE436': 'Tot. assets', 'SE441': 'Fixed Assets', 'SE446': 'Land etc',
         'SE450': 'Buildings', 'SE455': 'Machinery', 'SE460': 'Br.liv', 'SE465': 'Tot. cur. assets',
         'SE470': 'Non-br. liv', 'SE475': 'Prod. stock', 'SE480': 'Oth. circul. ',
         'SE600': 'Bal.Subs&Taxes', 'SE605': 'Subsidies'}

colors_out = ['#C70039', '#66DDF0']
colors_in = ['#66DDF0', '#C70039' ]
inputs = ['Total Hours', 'Total Area', 'Total Costs', 'Depreciation', 'Total Assets', 'Machinery']
outputs = ['T.Output/T.Costs', 'Crop Output', 'Cr.Out/Ha', 'Livestock Output', 'Lv.Out/L.Uni', 'Family Income', 'Subsidies']
ranks= {}
sygr = {}

In [ ]:
year1 = pnw.Select(name='Year', options=list(df0.YEAR.unique()))
perif1  = pnw.Select(name='Region', options=sorted(list(df0.REGION.unique())))
tf1  = pnw.Select(name='TF14', options=list(df0.TF14.unique()))

@pn.depends(perif1.param.value, tf1.param.value, year1.param.value )
def assets1(perif,tf,year):
    bal = df[choose + cols_bal]
    inc = df[choose + cols_inc]
    com = df[choose + cols_com]
    counter = df.YEAR[(df['REGION'] == perif) & (df['TF14'] == tf) & (df['YEAR'] == int(year))].count()
    if counter == 1:
        reg = com[(com['REGION'] == perif) & (com['TF14'] == tf) & (com['YEAR'] == int(year))]
        frame = com[(com['TF14'] == tf) & (com['YEAR'] == int(year)) & (com['REGION'] != 'All')]
        for var in cols_com:
            reg_val = reg[var].values[0]
            ranked_list = list(frame[var])
            ranked_list.append(reg_val)
            ranked_list = sorted(ranked_list)
            ranks[names[var]] = ranked_list.index(reg_val) + 1
            mesos, typikh = frame[var].mean(), frame[var].std()
            if typikh !=0:
                sygr[names[var]] = (reg_val - mesos) / typikh
            else:
                sygr[names[var]] = 0
        scatter0 = hv.Scatter((sygr))
        spikes = hv.Spikes(scatter0)
        eis = spikes[inputs].redim.range(y=(-2,2)) * scatter0[inputs]
        eis.opts(
            opts.Scatter(height=300, width=350, size=20, fill_color='#949194', tools=['hover']),
            opts.Spikes(color=dim('y').norm(), cmap=colors_in, line_width=20, invert_axes=True))
        eja =  spikes[outputs].redim.range(y=(-2,2)) * scatter0[outputs]
        eja.opts(
            opts.Scatter(height=300, width=350,size=20, fill_color='#949194', tools=['hover']),
            opts.Spikes(color=dim('y').norm(), cmap=colors_out, line_width=20, invert_axes=True))
        pre_lay = eis.relabel('Less is better').redim.label(y="Std's", x='Variable') + \
              eja.relabel('More is better').redim.label(y="Std's", x='Variable')
        pre_lay.opts(shared_axes=False, toolbar='right')
        lay = pn.Row(pre_lay)
    else:
        ign = "Doesn't exist"
        ign1 = "!!!!"
        ign2 = "%%%"
        lay = pn.Row(ign , ign1 , ign2)
    return lay


widgets1 = pn.Row("<br>\n###Select Region/Type of Crop/Year", perif1, tf1, year1, width=1400, background='#CCDED5')
ign = pn.Column(widgets1, assets1)

ign.servable()